In [33]:
!pip3 install cryptocompare

In [48]:
import cryptocompare
import requests
import json
import pandas as pd
import os
from time import sleep
from datetime import datetime
from dotenv import load_dotenv

In [49]:
load_dotenv()
apikey = os.getenv('apikey')
cryptocompare_api = os.getenv('cryptocompare_api')

In [83]:
def getPublicKeyTransactions():
    userInput = input('Choose profile:\n'
                  '1. Binance\n'
                  '2. Uniswap\n'
                  '3. Gemini\n')
    
    def getAddress(arg):
        addressBook = {1:'0x3f5CE5FBFe3E9af3971dD833D26bA9b5C936f0bE',
                       2:'0x88599C44e8EeB2Bbc2C1EB4749D37E09c8282d55',
                       3:'0xd24400ae8BfEBb18cA49Be86258a3C749cf46853'}
        return addressBook.get(arg)
              
    address = getAddress(int(userInput))
    
    getAddressTransactions = f'https://api.ethplorer.io/getAddressTransactions/{address}?limit=1000&apiKey={apikey}'
    getAddressTransactionsResponse = requests.get(getAddressTransactions).json()
    
    timestamp = []
    converted_timestamp = []
    hash_num = []
    value = []
    from_add = []
    to_add = []
    
    for transaction in getAddressTransactionsResponse:
        timestamp.append(transaction['timestamp'])
        converted_timestamp.append(datetime.fromtimestamp(transaction['timestamp']))
        hash_num.append(transaction['hash'])
        value.append(transaction['value'])
        from_add.append(transaction['from'])
        to_add.append(transaction['to'])

    df = pd.DataFrame({'converted_timestamp':converted_timestamp})
    ts = df['converted_timestamp']

    def roundNearestMinute(ts):
        return ts.dt.round('1min')

    rounded_timestamp = roundNearestMinute(ts)
    df = pd.DataFrame({'timestamp': timestamp, 'compare_timestamp': rounded_timestamp, 'transaction_hash': hash_num, 'value_in_eth': value, 'from_address': from_add, 'to_address': to_add})

    return df
    
pk_df = getPublicKeyTransactions()

Choose profile:
1. Binance
2. Uniswap
3. Gemini
 1


In [84]:
def getHistoricalPrice():
    set_crypto_key = f'cryptocompare.cryptocompare._set_api_key_parameter{cryptocompare_api}'
    getHistETHPrice = cryptocompare.get_historical_price_minute('ETH', currency='USD', limit=1440)

    timestamp = []
    converted_timestamp = []
    close = []

    for time in getHistETHPrice:
        timestamp.append(time['time'])
        converted_timestamp.append(datetime.fromtimestamp(time['time']))
        close.append(time['close'])

    df = pd.DataFrame({'timestamp':timestamp, 'compare_timestamp':converted_timestamp, 'close':close}).sort_values(by=['compare_timestamp'], ascending=False)
    return df

hp_df = getHistoricalPrice()

In [86]:
final_df = pd.merge(pk_df, hp_df, on='compare_timestamp')#.sort_values(by=['compare_timestamp'], ascending=True)
final_df.to_csv('getTransactionData.csv')
final_df.head()

,timestamp_x,compare_timestamp,transaction_hash,value_in_eth,from_address,to_address,timestamp_y,close
0,1620700989,2021-05-10 22:43:00,0x05a822c190a760aaf1ebd34cb928abbb5ec23bdd9c34...,0.285842,0x3f5ce5fbfe3e9af3971dd833d26ba9b5c936f0be,0xe434adf2341b85892b35756806078a5ac24e3e62,1620700980,3850.24
1,1620700979,2021-05-10 22:43:00,0xe6f84cacc57d6bab3b2f0297f57c2755063dca777fc6...,0.042320,0x3f5ce5fbfe3e9af3971dd833d26ba9b5c936f0be,0x6666668f5f19a2b5a97ce45b0830dce90ee932b7,1620700980,3850.24
2,1620700979,2021-05-10 22:43:00,0x53a53451592f82cffc9dff8b05cefd777af578c90e84...,0.042234,0x3f5ce5fbfe3e9af3971dd833d26ba9b5c936f0be,0x54edfb0af03305a92fd95e9a62b1284ff16f38c8,1620700980,3850.24
3,1620700979,2021-05-10 22:43:00,0x7e481a7c71171035bbb5ff1307c86f8e912f52814e71...,0.022899,0x3f5ce5fbfe3e9af3971dd833d26ba9b5c936f0be,0xe364384b7b0a0d77c79d333479ded1e31ce1ed78,1620700980,3850.24
4,1620700979,2021-05-10 22:43:00,0x369466a7f2f96f180d2e382d175e48f30dfe37ec1537...,0.103461,0x3f5ce5fbfe3e9af3971dd833d26ba9b5c936f0be,0xc4a7d387d2829fec54f16a763d8b80b61ac140aa,1620700980,3850.24
